In [7]:
import tensorflow as tf 
from create_sentiment_featuresets import create_feature_sets_and_labels
import pickle
import numpy as np

train_x,train_y,test_x,test_y = create_feature_sets_and_labels('data/pos.txt', 'data/neg.txt')

n_nodes_hl1 = 1500
n_nodes_hl2 = 1500
n_nodes_hl3 = 1500

n_classes = 2
batch_size = 100
epochs = 10


x = tf.placeholder('float')
y = tf.placeholder('float')


hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weights':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                  'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

output_layer = {'f_fum':None,
                'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'biases':tf.Variable(tf.random_normal([n_classes])),}

def neural_network_model(data):

    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer ['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer ['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer ['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3, output_layer['weights']), output_layer ['biases'])
    return output


def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            epoch_loss = 0
            
            i = 0
            while i < len(train_x):
                start = i
                end = i + batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                epoch_loss += c
                i += batch_size
            
            
            
            print('Epoch',epoch, 'completed out of ', epochs, 'loss: ', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
        
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        
        print('Accuracy: ', accuracy.eval({x:test_x, y:test_y}))




train_neural_network(x)





423
Epoch 0 completed out of  10 loss:  3835396.408187866
Epoch 1 completed out of  10 loss:  893414.0367774963
Epoch 2 completed out of  10 loss:  720883.7584514618
Epoch 3 completed out of  10 loss:  578743.6367969513
Epoch 4 completed out of  10 loss:  463170.88581609726
Epoch 5 completed out of  10 loss:  375248.22137498856
Epoch 6 completed out of  10 loss:  288615.15155792236
Epoch 7 completed out of  10 loss:  235433.79909324646
Epoch 8 completed out of  10 loss:  188210.95604395866
Epoch 9 completed out of  10 loss:  149791.0203356743
Accuracy:  0.8676311
